In [17]:
import networkx as nx
import numpy as np
import heapq
import unittest
from collections import deque
import matplotlib.pyplot as plt

In [12]:

def path(previous, s):
    '''
    `previous` is a dictionary chaining together the predecessor state that led to each state
    `s` will be None for the initial state
    otherwise, start from the last state `s` and recursively trace `previous` back to the initial state,
    constructing a list of states visited as we go
    '''
    if s is None:
        return []
    else:
        return path(previous, previous[s])+[s]

def pathcost(path, step_costs):
    '''
    add up the step costs along a path, which is assumed to be a list output from the `path` function above
    '''
    cost = 0
    for s in range(len(path)-1):
        cost += step_costs[path[s]][path[s+1]]
    return cost

map_distances = dict(
    chi=dict(det=283, cle=345, ind=182),
    cle=dict(chi=345, det=169, col=144, pit=134, buf=189),
    ind=dict(chi=182, col=176),
    col=dict(ind=176, cle=144, pit=185),
    det=dict(chi=283, cle=169, buf=256),
    buf=dict(det=256, cle=189, pit=215, syr=150),
    pit=dict(col=185, cle=134, buf=215, phi=305, bal=247),
    syr=dict(buf=150, phi=253, new=254, bos=312),
    bal=dict(phi=101, pit=247),
    phi=dict(pit=305, bal=101, syr=253, new=97),
    new=dict(syr=254, phi=97, bos=215, pro=181),
    pro=dict(bos=50, new=181),
    bos=dict(pro=50, new=215, syr=312, por=107),
    por=dict(bos=107))

forward_providence = dict(
    chi=833,
    cle=531,
    ind=782,
    col=618,
    det=596,
    buf=385,
    pit=458,
    syr=253,
    bal=325,
    phi=236,
    new=157,
    pro=0,
    bos=38,
    por=136)

backward_providence = dict(
    chi=0,
    cle=202,
    ind=75,
    col=215,
    det=350,
    buf=500,
    pit=450,
    syr=635,
    bal=575,
    phi=600,
    new=675,
    pro=833,
    bos=850,
    por=915)
# Solution:

def forward_heuristic_sld_providence(state):
    return forward_providence[state]
def backward_heuristic_sld_providence(state):
    return backward_providence[state]

class Frontier_PQ:
    ''' frontier class for uniform search, ordered by path cost '''
    def __init__(self,start,cost):
        self.states = {}
        self.q = []
        self.add(start,cost)
    def add(self,state,cost):
        heapq.heappush(self.q,(cost,state))
        self.states[state]=cost
    def pop(self):
        (cost,state) = heapq.heappop(self.q)
        self.states.pop(state)
        return (cost,state)
    def replace(self,state,cost):
        if(self.states[state] > cost):
            self.states[state] = cost
        for i,(currentCost,currentState) in enumerate(self.q):
            if (currentState == state and currentCost > cost):
                self.q[i] = (cost,state)
                heapq.heapify(self.q)
                return
    def contains(self, node):
        return node in self.states

def astar_search(start, goal, state_graph, forwardHeuristic, backwardHeuristic,return_cost=False, return_nexp=False):
    '''A* search from `start` to `goal`
    start = initial state
    goal = goal state
    heuristic = function for estimated cost to goal (function name)
    return_cost = logical (True/False) for whether or not to return the total path cost
    return_nexp = logical (True/False) for whether or not to return the number of nodes expanded
    '''
    frontStep = 0
    backStep = 0
    frontFrontier = Frontier_PQ(start,0)
    backFrontier = Frontier_PQ(goal,0)
    frontVisited={}
    backVisited = {}
    previous={start:None}
    next = {goal:None}
    while frontFrontier and backFrontier:
        if frontFrontier:
            frontStep +=1
            #print(frontStep)
            Fstate = frontFrontier.pop()
            #print(Fstate)
            #print(Fstate[1] == goal or backFrontier.contains(Fstate[1]))
            if Fstate[1] == goal or backFrontier.contains(Fstate[1]):
                #print(backFrontier.contains(Fstate[1]))
                if return_cost:
                    if return_nexp:
                        return path(previous,Fstate[1]),Fstate[0],nexp
                    return path(previous,Fstate[1]),Fstate[0]
                else:
                    if return_nexp:
                        return path(previous,Fstate[1]),None,nexp
                    return path(previous,Fstate[1]), path(next,Bstate[1])
            frontVisited[Fstate[1]] = pathcost(path(previous,Fstate[1]), state_graph)
            for nextState in state_graph[Fstate[1]]:
                updateCost = frontVisited[Fstate[1]]+state_graph[Fstate[1]][nextState]+forwardHeuristic(nextState)
                if( nextState not in frontVisited and nextState not in frontFrontier.states):
                    frontFrontier.add(nextState,updateCost)
                    previous[nextState] = Fstate[1]
                elif( nextState in frontFrontier.states and frontFrontier.states[nextState] > updateCost ):
                    frontFrontier.replace(nextState, updateCost)
                    previous[nextState] = Fstate[1]
        if backFrontier:
            backStep += 1
            #print(backStep)
            Bstate = backFrontier.pop()
            if Bstate[1] == start or frontFrontier.contains(Bstate[1]):
                #print("back")
                if return_cost:
                    if return_nexp:
                        return path(next,Bstate[1]),Bstate[0],nexp
                    return path(next,Bstate[1]),Bstate[0]
                else:
                    if return_nexp:
                        return path(next,Bstate[1]),None,nexp
                    return path(previous,Fstate[1]),path(next,Bstate[1])
            backVisited[Bstate[1]] = pathcost(path(next,Bstate[1]), state_graph)
            for prevState in state_graph[Bstate[1]]:
                updateCost = backVisited[Bstate[1]]+state_graph[Bstate[1]][prevState]+backwardHeuristic(nextState)
                if( prevState not in backVisited and prevState not in backFrontier.states):
                    backFrontier.add(prevState,updateCost)
                    next[prevState] = Bstate[1]
                elif( prevState in backFrontier.states and backFrontier.states[prevState] > updateCost ):
                    backFrontier.replace(prevState, updateCost)
                    next[prevState] = Bstate[1]

path1,path2 = astar_search('chi','pro', map_distances, forward_heuristic_sld_providence, backward_heuristic_sld_providence, return_cost=False, return_nexp=False)
print("Path:",path1,path2)
#print("Cost:",cost)



Path: ['chi', 'cle', 'pit'] ['pro', 'new', 'phi']


In [15]:
graph = nx.Graph()
nodes = [
    'chi',
    'cle',
    'ind',
    'col',
    'det',
    'buf',
    'pit',
    'syr',
    'bal',
    'phi',
    'new',
    'pro',
    'bos',
    'por']
edges = [('chi','det'), ('chi','cle'), ('chi','ind'),
    ('cle','chi'), ('cle','det'), ('cle','col'), ('cle','pit'), ('cle','buf'),
    ('ind','chi'), ('ind','col'),
    ('col','ind'), ('col','cle'), ('col','pit'),
    ('det','chi'), ('det','cle'), ('det','buf'),
    ('buf','det'), ('buf','cle'), ('buf','pit'), ('buf','syr'),
    ('pit','col'), ('pit','cle'), ('pit','buf'), ('pit','phi'), ('pit','bal'),
    ('syr','buf'), ('syr','phi'), ('syr','new'), ('syr','bos'),
    ('bal','phi'), ('bal','pit'),
    ('phi','pit'), ('phi','bal'), ('phi','syr'), ('phi','new'),
    ('new','syr'), ('new','phi'), ('new','bos'), ('new','pro'),
    ('pro','bos'), ('pro','new'),
    ('bos','pro'), ('bos','new'), ('bos','syr'), ('bos','por'),
    ('por','bos')]
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)
plt.subplot(421)

nx.draw(graph, with_labels=True, font_weight='bold')

In [50]:
graph['chi']

AtlasView({'det': {}, 'cle': {}, 'ind': {}})

In [48]:
plt.show()

In [51]:
G = nx.petersen_graph()
plt.subplot(721)
nx.draw(G, with_labels=True, font_weight='bold')

In [52]:
plt.show()